In [ ]:
# fmt: off
import copy
import os
import pickle
import shutil

import dask.dataframe as dd
import h5py
import matplotlib
import numpy as np
import pandas as pd
import scipy as sp
import skimage as sk
from scipy import ndimage as ndi

matplotlib.rcParams["figure.figsize"] = [20, 10]

from time import sleep

import scipy.ndimage.morphology as morph
# import mahotas as mh
from dask.distributed import worker_client
from matplotlib import pyplot as plt
from pandas import HDFStore
from skimage import (exposure, feature, filters, future, measure, morphology,
                     segmentation, transform, util)
from skimage.segmentation import watershed

from paulssonlab.deaton.trenchripper.trenchripper import (hdf5lock,
                                                          kymo_handle,
                                                          pandas_hdf5_handler,
                                                          writedir)

In [ ]:
with h5py.File(
    "/n/scratch3/users/d/de64/190922_20x_phase_gfp_segmentation/190922_20x_phase_gfp_segmentation/kymograph/kymograph_10.hdf5",
    "r",
) as infile:
    data = infile["GFP"][:]

In [ ]:
a = sk.transform.rescale(data[12, 0], 2.0)

In [ ]:
plt.imshow(a)

In [ ]:
hist_range = (0, np.percentile(a.flatten(), 90))
freq, val = np.histogram(a.flatten(), bins=200, range=hist_range)
mu_n = val[np.argmax(freq)]

lower_tail = a[a < mu_n].flatten()
std_n = halfnorm.fit(lower_tail)[1]

In [ ]:
b = sk.filters.sobel(a)

In [ ]:
plt.imshow(b)

In [ ]:
from skimage.feature import peak_local_max
from skimage.segmentation import watershed

In [ ]:
local_maxi = peak_local_max(-b, indices=False, footprint=np.ones((10, 10)))
markers = ndi.label(local_maxi)[0]

In [ ]:
plt.imshow(markers)

In [ ]:
labels = watershed(b, markers=markers)

In [ ]:
plt.imshow(labels)

In [ ]:
output = np.zeros(labels.shape, dtype=float)
for i in np.unique(labels).tolist():
    med_val = np.median(a[labels == i])
    output[labels == i] = med_val

In [ ]:
plt.imshow(output)

In [ ]:
bound = sk.segmentation.find_boundaries(labels, mode="thick")

In [ ]:
plt.imshow(bound)

In [ ]:
selem = sk.morphology.disk(3)
decile = sk.filters.rank.percentile(a, selem, p0=0.1)

In [ ]:
plt.imshow(decile)

In [ ]:
decile[~bound] = 0
plt.imshow(decile)

In [ ]:
sk.filters.laplace(eximg, ksize=4)

In [ ]:
plt.imshow(eximg > background_thr)

In [ ]:
background_thr

In [ ]:
mu_n = val[np.argmax(freq)]

In [ ]:
lower_tail = eximg[eximg < mu_n].flatten()
std_n = halfnorm.fit(lower_tail)[1]

In [ ]:
plt.hist(eximg.flatten())

In [ ]:
from scipy.stats import halfnorm

In [ ]:
halfnorm.fit(lower_tail)[1]

### Fluorescence Heterogeneity

In [ ]:
# fmt: off
import copy
import os
import pickle
import shutil

import dask.dataframe as dd
import h5py
import matplotlib
import numpy as np
import pandas as pd
import scipy as sp
import skimage as sk
from scipy import ndimage as ndi

matplotlib.rcParams["figure.figsize"] = [20, 10]

from time import sleep

import scipy.ndimage.morphology as morph
# import mahotas as mh
from dask.distributed import worker_client
from matplotlib import pyplot as plt
from pandas import HDFStore
from skimage import (exposure, feature, filters, future, measure, morphology,
                     segmentation, transform, util)
from skimage.segmentation import watershed

import paulssonlab.deaton.trenchripper.trenchripper as tr
from paulssonlab.deaton.trenchripper.trenchripper import (hdf5lock,
                                                          kymo_handle,
                                                          pandas_hdf5_handler,
                                                          writedir)

#### Example 1

In [ ]:
with h5py.File(
    "/n/scratch3/users/d/de64/190922_20x_phase_gfp_segmentation/190922_20x_phase_gfp_segmentation/kymograph/kymograph_10.hdf5",
    "r",
) as infile:
    data = infile["GFP"][:]

In [ ]:
example_1 = data[2, :10]

In [ ]:
# with h5py.File("/home/de64/paulssonlab/paulssonlab/src/paulssonlab/deaton/trenchripper/dev_notebooks/2020-06-19_revisiting_training_data_assembly/example_1.hdf5","w") as outfile:
#     hdf5_dataset = outfile.create_dataset("data", data=example_1, dtype="uint16")

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(example_1)
plt.imshow(handle.return_unwrap())

In [ ]:
class fluo_segmentation:
    def __init__(self,bit_max=0,scale_timepoints=False,scaling_percentile=0.9,img_scaling=1.,smooth_sigma=0.75,niblack_scaling=1.,\
                 hess_pad=6,global_threshold=25,triangle_threshold_scaling=1.,cell_otsu_scaling=1.,local_otsu_r=15,min_obj_size=30,distance_threshold=2):

In [ ]:
current_seg = tr.fluo_segmentation(
    bit_max=10000,
    img_scaling=1.0,
    smooth_sigma=1.0,
    global_threshold=40,
    distance_threshold=1,
    local_otsu_r=5,
    triangle_threshold_scaling=1.0,
)

In [ ]:
output = current_seg.segment(example_1)

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(output)
plt.imshow(handle.return_unwrap())

In [ ]:
img_arr = example_1

t_tot = img_arr.shape[0]
img_arr = current_seg.to_8bit(img_arr, current_seg.bit_max)
if current_seg.scale_timepoints:
    img_arr = current_seg.scale_kymo(img_arr, current_seg.scaling_percentile)

input_kymo = kymo_handle()
input_kymo.import_wrap(img_arr)
del img_arr

input_kymo = input_kymo.return_unwrap()
original_shape = input_kymo.shape
input_kymo = transform.rescale(
    input_kymo, current_seg.img_scaling, anti_aliasing=False, preserve_range=True
).astype("uint8")
input_kymo = sk.filters.gaussian(
    input_kymo, sigma=current_seg.smooth_sigma, preserve_range=True, mode="reflect"
).astype("uint8")

eig_img = current_seg.get_eig_img(input_kymo, edge_padding=current_seg.hess_pad)
eig_mask = current_seg.get_eig_mask(eig_img, hess_thr_scale=current_seg.hess_thr_scale)
# del eig_img

cell_mask = current_seg.get_cell_mask(
    input_kymo,
    global_threshold=current_seg.global_threshold,
    triangle_threshold_scaling=current_seg.triangle_threshold_scaling,
    cell_otsu_scaling=current_seg.cell_otsu_scaling,
    local_otsu_r=current_seg.local_otsu_r,
    min_obj_size=current_seg.min_obj_size,
)
del input_kymo

# dist_img = ndi.distance_transform_edt(cell_mask).astype("uint8")
# dist_mask = dist_img>self.distance_threshold
# marker_mask = dist_mask*eig_mask
# del dist_mask
# marker_mask = sk.measure.label(marker_mask)

# output_labels = watershed(-dist_img, markers=marker_mask, mask=cell_mask)

# del dist_img
# del marker_mask
# del cell_mask
# output_labels = sk.transform.resize(output_labels,original_shape,order=0,anti_aliasing=False, preserve_range=True).astype("uint32")

# output_kymo = kymo_handle()
# output_kymo.import_unwrap(output_labels,t_tot)
# del output_labels
# output_kymo = output_kymo.return_wrap()
# for i in range(output_kymo.shape[0]):
#     output_kymo[i] = self.reorder_ids(output_kymo[i])
# return output_kymo

In [ ]:
plt.imshow(cell_mask)

In [ ]:
eig_thr = sk.filters.threshold_niblack(eig_img)
eig_mask = eig_img > eig_thr

In [ ]:
plt.imshow(eig_img)

In [ ]:
eig_thr = sk.filters.threshold_niblack(eig_img) * 1.2
eig_mask = eig_img > eig_thr

In [ ]:
plt.imshow(eig_mask)

In [ ]:
plt.imshow(sk.filters.sobel(eig_img))

In [ ]:
plt.hist(eig_img.flatten(), bins=30)
plt.show()

In [ ]:
import scipy as sp

In [ ]:
hess_thr_scale = 1.0
hist_range = (0, np.percentile(eig_img.flatten(), 90))
freq, val = np.histogram(eig_img.flatten(), bins=200, range=hist_range)
mu_n = val[np.argmax(freq)]

upper_tail = eig_img[eig_img > mu_n].flatten()
std_n = sp.stats.halfnorm.fit(upper_tail)[1]
background = mu_n - hess_thr_scale * std_n

In [ ]:
plt.imshow(eig_img > background)

#### Example 2

In [ ]:
with h5py.File(
    "/n/scratch3/users/d/de64/190925_20x_phase_yfp_segmentation/190925_20x_phase_yfp_segmentation/kymograph/kymograph_10.hdf5",
    "r",
) as infile:
    data = infile["YFP"][:]

In [ ]:
example_2 = data[0, :10]

In [ ]:
# with h5py.File("/home/de64/paulssonlab/paulssonlab/src/paulssonlab/deaton/trenchripper/dev_notebooks/2020-06-19_revisiting_training_data_assembly/example_2.hdf5","w") as outfile:
#     hdf5_dataset = outfile.create_dataset("data", data=example_2, dtype="uint16")

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(example_2)
plt.imshow(handle.return_unwrap())

In [ ]:
current_seg = tr.fluo_segmentation(
    bit_max=10000,
    img_scaling=1.0,
    smooth_sigma=1.0,
    global_threshold=30,
    distance_threshold=1,
    hess_thr_scale=0.4,
    local_otsu_r=7,
)

In [ ]:
output = current_seg.segment(example_2)

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(output)
plt.imshow(handle.return_unwrap())

In [ ]:
img_arr = example_2

t_tot = img_arr.shape[0]
img_arr = current_seg.to_8bit(img_arr, current_seg.bit_max)
if current_seg.scale_timepoints:
    img_arr = current_seg.scale_kymo(img_arr, current_seg.scaling_percentile)

input_kymo = kymo_handle()
input_kymo.import_wrap(img_arr)
del img_arr

input_kymo = input_kymo.return_unwrap()
original_shape = input_kymo.shape
input_kymo = transform.rescale(
    input_kymo, current_seg.img_scaling, anti_aliasing=False, preserve_range=True
).astype("uint8")
input_kymo = sk.filters.gaussian(
    input_kymo, sigma=current_seg.smooth_sigma, preserve_range=True, mode="reflect"
).astype("uint8")

eig_img = current_seg.get_eig_img(input_kymo, edge_padding=current_seg.hess_pad)
eig_mask = current_seg.get_eig_mask(eig_img, hess_thr_scale=current_seg.hess_thr_scale)
# del eig_img

cell_mask = current_seg.get_cell_mask(
    input_kymo,
    global_threshold=current_seg.global_threshold,
    triangle_threshold_scaling=current_seg.triangle_threshold_scaling,
    cell_otsu_scaling=current_seg.cell_otsu_scaling,
    local_otsu_r=current_seg.local_otsu_r,
    min_obj_size=current_seg.min_obj_size,
)
del input_kymo

In [ ]:
plt.imshow(eig_img)

In [ ]:
plt.imshow(eig_mask)

#### Example 3

In [ ]:
with h5py.File(
    "/n/scratch3/users/d/de64/Vibrio_2_1_TOP/Vibrio_2_1_TOP/kymograph/kymograph_4.hdf5",
    "r",
) as infile:
    print(infile.keys())
    data = infile["GFP"][:]

In [ ]:
example_3 = data[5, :7]

In [ ]:
# with h5py.File("/home/de64/paulssonlab/paulssonlab/src/paulssonlab/deaton/trenchripper/dev_notebooks/2020-06-19_revisiting_training_data_assembly/example_3.hdf5","w") as outfile:
#     hdf5_dataset = outfile.create_dataset("data", data=example_3, dtype="uint16")

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(example_3)
plt.imshow(handle.return_unwrap())
img_arr = handle.return_unwrap()

In [ ]:
def to_8bit(img_arr, bit_max=None):
    img_max = np.max(img_arr) + 0.0001
    if bit_max is None:
        max_val = img_max
    else:
        max_val = max(img_max, bit_max)
    min_val = np.min(img_arr)
    #         min_val = np.min(img_arr)
    norm_array = (img_arr - min_val) / (max_val - min_val)
    norm_byte_array = sk.img_as_ubyte(norm_array)
    return norm_byte_array


def get_eig_img(img_arr, edge_padding=6):
    inverted = sk.util.invert(img_arr)
    del img_arr
    inverted = np.pad(inverted, edge_padding, "reflect")
    hessian = sk.feature.hessian_matrix(inverted, order="rc")
    del inverted
    eig_img = sk.feature.hessian_matrix_eigvals(hessian)
    del hessian
    eig_img = np.min(eig_img, axis=0)
    eig_img = eig_img[edge_padding:-edge_padding, edge_padding:-edge_padding]
    max_val, min_val = np.max(eig_img), np.min(eig_img)
    eig_img = to_8bit(eig_img)
    return eig_img


def get_eig_mask(eig_img, hess_thr_scale=1.0):
    hist_range = (0, np.percentile(eig_img.flatten(), 90))
    freq, val = np.histogram(eig_img.flatten(), bins=50, range=hist_range)
    mu_n = val[np.argmax(freq)]

    upper_tail = eig_img[eig_img > mu_n].flatten()
    std_n = sp.stats.halfnorm.fit(upper_tail)[1]
    background = mu_n - (hess_thr_scale * std_n)
    eig_mask = eig_img > background
    return eig_mask

In [ ]:
eig_img = get_eig_img(img_arr)

In [ ]:
eig_mask = get_eig_mask(eig_img)

In [ ]:
plt.imshow(-eig_img)

In [ ]:
dist_img = ndi.distance_transform_edt(cell_mask).astype("uint8")
dist_mask = dist_img > self.distance_threshold
marker_mask = dist_mask * eig_mask
del dist_mask
marker_mask = sk.measure.label(marker_mask)

In [ ]:
def get_background_dist(array, min_tail=30):
    hist_range = (0, np.percentile(array.flatten(), 90))
    freq, val = np.histogram(array.flatten(), bins=50, range=hist_range)
    mu_n = val[np.argmax(freq)]
    lower_tail = array[array < mu_n].flatten()
    if len(lower_tail) > min_tail:
        std_n = sp.stats.halfnorm.fit(lower_tail)[1]
    else:
        std_n = np.std(array.flatten())
    return mu_n, std_n

In [ ]:
mu_n, std_n = get_background_dist(sobel_img)
bkd = mu_n + 20 * std_n

In [ ]:
matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
sobel_img = sk.filters.sobel(handle.return_unwrap())
plt.imshow(sobel_img)
plt.show()

In [ ]:
plt.hist(handle.return_unwrap().flatten(), bins=30)

In [ ]:
foreground_mask = handle.return_unwrap() > sk.filters.threshold_niblack(
    handle.return_unwrap(), window_size=21, k=0.01
)
plt.imshow(foreground_mask)
plt.show()
foreground_mask = ~sk.morphology.binary_closing(~foreground_mask)
plt.imshow(foreground_mask)
plt.show()
foreground_mask = sk.morphology.remove_small_objects(foreground_mask, min_size=150)
plt.imshow(foreground_mask)
plt.show()

In [ ]:
plt.hist(img_arr[~cell_mask].flatten(), bins=30, range=(0, 4000))
plt.show()

In [ ]:
plt.hist(img_arr[cell_mask].flatten(), bins=30, range=(0, 4000))
plt.show()

In [ ]:
back_pixels = img_arr[~cell_mask].flatten()

In [ ]:
hess_thr_scale = 0.5
hist_range = (0, np.percentile(back_pixels, 90))
freq, val = np.histogram(back_pixels, bins=100, range=hist_range)
mu_n = val[np.argmax(freq)]

lower_tail = back_pixels[back_pixels < mu_n].flatten()
std_n = sp.stats.halfnorm.fit(lower_tail)[1]
background = mu_n + hess_thr_scale * std_n

In [ ]:
mu_n

In [ ]:
plt.imshow(img_arr > background)

In [ ]:
otsu_selem = sk.morphology.disk(7)
otsu_thr = sk.filters.rank.otsu(img_arr, otsu_selem) * 1.0
# del otsu_selem
# thr = np.maximum(tri_thr,otsu_thr)


# cell_mask = (img_arr>thr)
# plt.imshow(cell_mask)
# plt.show()
# cell_mask = (img_arr>tri_thr)
# plt.imshow(cell_mask)
# plt.show()
cell_mask = img_arr > otsu_thr
plt.imshow(cell_mask)
plt.show()

In [ ]:
plt.imshow(cell_mask)

In [ ]:
plt.imshow(
    handle.return_unwrap() > sk.filters.threshold_niblack(handle.return_unwrap())
)

In [ ]:
plt.hist(example_3.flatten())

In [ ]:
current_seg = tr.fluo_segmentation(
    bit_max=5000,
    img_scaling=1.0,
    smooth_sigma=1.0,
    global_threshold=35,
    distance_threshold=1,
    hess_thr_scale=1.0,
    local_otsu_r=5,
    triangle_threshold_scaling=1.1,
    min_obj_size=150,
)

In [ ]:
output = current_seg.segment(example_3)

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(output)
plt.imshow(handle.return_unwrap()[:, :])

In [ ]:
img_arr = example_3

t_tot = img_arr.shape[0]
img_arr = current_seg.to_8bit(img_arr, current_seg.bit_max)
if current_seg.scale_timepoints:
    img_arr = current_seg.scale_kymo(img_arr, current_seg.scaling_percentile)

input_kymo = kymo_handle()
input_kymo.import_wrap(img_arr)

input_kymo = input_kymo.return_unwrap()
original_shape = input_kymo.shape
input_kymo = transform.rescale(
    input_kymo, current_seg.img_scaling, anti_aliasing=False, preserve_range=True
).astype("uint8")
input_kymo = sk.filters.gaussian(
    input_kymo, sigma=current_seg.smooth_sigma, preserve_range=True, mode="reflect"
).astype("uint8")

eig_img = current_seg.get_eig_img(input_kymo, edge_padding=current_seg.hess_pad)
eig_mask = current_seg.get_eig_mask(eig_img, hess_thr_scale=current_seg.hess_thr_scale)
# del eig_img

# cell_mask = current_seg.get_cell_mask(input_kymo,global_threshold=current_seg.global_threshold,\
#             triangle_threshold_scaling = current_seg.triangle_threshold_scaling,\
#             cell_otsu_scaling=current_seg.cell_otsu_scaling,local_otsu_r=current_seg.local_otsu_r,\
#             min_obj_size=current_seg.min_obj_size)

In [ ]:
def get_background_dist(array):
    hist_range = (0, np.percentile(array.flatten(), 90))
    freq, val = np.histogram(array.flatten(), bins=100, range=hist_range)
    mu_n = val[np.argmax(freq)]

    lower_tail = array[array < mu_n].flatten()
    std_n = sp.stats.halfnorm.fit(lower_tail)[1]
    return mu_n, std_n

In [ ]:
from skimage.segmentation import clear_border

In [ ]:
def get_background_dist(self,array):
    hist_range = (0,np.percentile(array.flatten(),90))
    freq,val = np.histogram(array.flatten(),bins=100,range=hist_range)
    mu_n = val[np.argmax(freq)]

    lower_tail = array[array<mu_n].flatten()
    std_n = sp.stats.halfnorm.fit(lower_tail)[1]
    return mu_n,std_n

def get_cell_mask(self,input_kymo,niblack_window_size = 15,min_obj_size = 30, background_scaling = 1., global_threshold = 0)

    wrapped_kymo = kymo_handle()
    wrapped_kymo.import_unwrap(input_kymo,t_tot)
    wrapped_kymo = wrapped_kymo.return_wrap()

    wrapped_cell_mask = []

    for t in range(wrapped_kymo.shape[0]):
        img_arr = wrapped_kymo[t]
        cell_mask = img_arr>sk.filters.threshold_niblack(img_arr, window_size=niblack_window_size)
        cell_mask = sk.morphology.remove_small_objects(cell_mask,min_size=min_obj_size)

        labeled_mask = sk.morphology.label(cell_mask)
        del cell_mask

        background = img_arr[labeled_mask==0]
        mu_n,std_n = self.get_background_dist(background)
        del background

        try:

            rps = sk.measure.regionprops(labeled_mask,intensity_image=img_arr)
            obj_intensities = np.array([rp.mean_intensity for rp in rps])
            del rps

            back_thr = mu_n + background_scaling*std_n
            init_cell_mask = obj_intensities>back_thr

            cell_list = np.where(init_cell_mask)[0]+1
            del init_cell_mask
            cell_mask = np.isin(labeled_mask,cell_list)
            del cell_list

            cell_mask = cell_mask*(img_arr>global_threshold)
            del img_arr

            cell_mask = sk.morphology.remove_small_holes(cell_mask)

        except:
            cell_mask = np.zeros(labeled_mask.shape,dtype=bool)
        del labeled_mask
        wrapped_cell_mask.append(cell_mask)
        del cell_mask


    del wrapped_kymo
    wrapped_cell_mask = np.array(wrapped_cell_mask)

    cell_mask = kymo_handle()
    cell_mask.import_wrap(wrapped_cell_mask)
    del wrapped_cell_mask
    cell_mask = cell_mask.return_unwrap()
    
    return cell_mask

In [ ]:

border_buffer = 1
        cell_mask = sk.segmentation.clear_border(cell_mask, buffer_size=border_buffer)


In [ ]:
current = tr.fluo_segmentation(
    bit_max=10000,
    scale_timepoints=False,
    scaling_percentile=0.9,
    img_scaling=1.0,
    smooth_sigma=2.0,
    hess_thr_scale=1.0,
    hess_pad=6,
    global_threshold=0,
    niblack_window_size=15,
    background_scaling=1.0,
    min_obj_size=150,
    distance_threshold=2,
)

In [ ]:
output = current.segment(example_3)

In [ ]:
handle = tr.kymo_handle()
handle.import_wrap(output)
plt.imshow(handle.return_unwrap())

In [ ]:
plt.imshow(input_kymo)

In [ ]:
plt.imshow(cell_mask)

In [ ]:
otsu_selem = sk.morphology.disk(5)
otsu_thr = sk.filters.rank.otsu(input_kymo, otsu_selem) * current_seg.cell_otsu_scaling
cell_mask = input_kymo > otsu_thr
labeled_mask = sk.morphology.label(cell_mask)

rps = sk.measure.regionprops(labeled_mask, intensity_image=input_kymo)
obj_intensities = np.array([rp.mean_intensity for rp in rps])
tri_thr = sk.filters.threshold_triangle(obj_intensities) * 1.0
cell_list = np.where(obj_intensities > tri_thr)[0] + 1
obj_mask = np.isin(labeled_mask, cell_list)
obj_mask = sk.morphology.remove_small_objects(obj_mask, min_size=100)

In [ ]:
sk.filters.threshold_triangle(input_kymo[cell_mask]) * 1.0

In [ ]:
plt.hist(input_kymo[cell_mask], bins=30)

In [ ]:
plt.hist(obj_intensities, bins=30)

In [ ]:
plt.imshow(obj_mask)

In [ ]:
plt.imshow(cell_mask * (input_kymo > 50))

In [ ]:
plt.hist(obj_intensities, bins=30)

In [ ]:
plt.hist(input_kymo.flatten(), bins=30, range=(0, 200))

In [ ]:
sobel_thr_scale = 1.0
hist_range = (0, np.percentile(input_kymo.flatten(), 90))
freq, val = np.histogram(input_kymo.flatten(), bins=100, range=hist_range)
mu_n = val[np.argmax(freq)]

lower_tail = input_kymo[input_kymo < mu_n].flatten()
std_n = sp.stats.halfnorm.fit(lower_tail)[1]
background = mu_n + sobel_thr_scale * std_n

In [ ]:
background

In [ ]:
mu_n

In [ ]:
sobel_img = sk.filters.sobel(input_kymo)

In [ ]:
plt.imshow(sobel_img)

In [ ]:
plt.hist(sobel_img.flatten(), bins=30)

In [ ]:
sobel_thr_scale = 5.0
hist_range = (0, np.percentile(sobel_img.flatten(), 90))
freq, val = np.histogram(sobel_img.flatten(), bins=100, range=hist_range)
mu_n = val[np.argmax(freq)]

lower_tail = sobel_img[sobel_img < mu_n].flatten()
std_n = sp.stats.halfnorm.fit(lower_tail)[1]
background = mu_n + sobel_thr_scale * std_n

sobel_mask = sk.filters.apply_hysteresis_threshold(
    sobel_img, mu_n + 5.0 * std_n, mu_n + 10.0 * std_n
)

# sobel_mask = sobel_img>background
sobel_mask = sk.morphology.remove_small_holes(sobel_mask, area_threshold=200)

In [ ]:
plt.imshow(sobel_img > np.percentile(sobel_img.flatten(), 60))

In [ ]:
plt.imshow(cell_mask * sobel_mask)

In [ ]:
background

In [ ]:
img_arr.shape

In [ ]:
otsu_selem = sk.morphology.disk(current_seg.local_otsu_r)
tri_thr = (
    sk.filters.threshold_triangle(input_kymo) * current_seg.triangle_threshold_scaling
)
otsu_thr = sk.filters.rank.otsu(input_kymo, otsu_selem) * current_seg.cell_otsu_scaling
thr = np.maximum(tri_thr, otsu_thr)
cell_mask = (input_kymo > thr) * (input_kymo > current_seg.global_threshold)
cell_mask = sk.morphology.remove_small_objects(
    cell_mask, min_size=current_seg.min_obj_size
)
cell_mask = sk.morphology.remove_small_holes(cell_mask)

In [ ]:
plt.imshow(cell_mask)